# Checkers

## Regole della dama
Da: https://www.federdama.org/cms/index.php/documenti-1/le-specialita/dama-italiana
- la damiera è di dimensione 8 X 8 e l'ultima casella in basso deve essere di colore nero
- ogni giocatore ha 12 pedine a testa
- le pedine si muovono sempre in diagonale sulle case scure, di una casa alla volta e sempre in avanti
- quando una pedina raggiunge l'altra estremità della scacchiera diventa dama
- ogni pedina *è obbligata* a mangiare quelle avversarie che si trovano davanti nella casella diagonale se la casella successiva è libera. Se ci sono ulteriori pedine e le caselle diagonali sono libere, si continua a mangiare (fino a 3 pezzi). Bisogna mangiare obbligatoriamente il maggior numero di pezzi
- è obbligatorio mangiare con il pezzo più forte (dama)

La profondità dell'albero di ricerca può essere modificata cambiando il valore
```
self.engine = Engine(3)
```
che si trova nel metodo `__init__` della classe `UI`. Nell'esempio sopra, la profondità è fissata a 3. Per fissarla a 2, per esempio, basterà sostituire il 3 con il 2. Notare: con profondità maggiore di 4 l'algoritmo rallenta molto.

Definiamo la scacchiera e lo stato come delle `namedtuple`. Vedere: https://realpython.com/python-namedtuple/.

In [3]:
Board = namedtuple('Board', 'cells')
State = namedtuple('State', 'board, to_move')

La scacchiera è rappresentata come una matrice (lista di liste) dove ciascuna entry è:

- `'x'` oppure `'o'` dove `'x'` è una pedina del giocatore nero e `'o'` una pedina del giocatore bianco
- `'-'` è una casella libera
- `'X'` e `'O'` (maiuscole) rappresentano le dame

La cella in alto a sinistra ha coordinate 0,0 mentre quella in basso a destra 7,7.

Lo stato del gioco è rappresentato tramite una coppia `(board, to_move)` dove `board` è una scacchiera come indicato sopra e `to_move` è `'x'` oppure `'o'` e rappresenta il giocatore che deve muovere.

### Classe Game

Definiamo una classe `Game` che rappresenta un gioco.

In [4]:
class Game:
    
    # metodo che controlla se lo stato corrente è terminale
    def is_terminal(self, state) -> bool:
        return self.actions(state) == []

    
    # metodo che restituisce il giocatore che deve muovere
    def to_move(self, state):
        return state.to_move

    
    '''
    Metodo che definisce l'utilità di uno stato dato un giocatore.
    Questo metodo viene utilizzato nell'algoritmo min-max e alpha-beta.
    Rappresentare l'utilità dello stato come si preferisce (ad esempio:
    numero di pedine che restano al giocatore tenendo conto che una dama
    è più importante di una pedina).
    '''
    def utility(self, state, player):
        # TODO: provare ad implementare altre utility
        mine = 0
        for row in state.board:
            for cell in row:
                if cell == player:
                    mine = mine + 1
                if cell == player.upper():
                    mine = mine + 2
                if cell == opponent(player):
                    mine = mine - 1
                if cell == opponent(player).upper():
                    mine = mine - 2
        return mine


    '''
    Metodo che restituisce una lista di azioni possibili dato uno stato.
    La azioni possibili sono rappresentate tramite una lista di tuple.
    Ci sono due possibilità:
    - spostamento pedina o dama ('move', (row, col), (delta_row, delta_col))
    - salto pedina o dama ('jump', (row, col), (delta_row, delta_col))
    Prima controllo se ci sono mosse possibili di salto perché se posso
    mangiare una pedina devo farlo; se non ci sono mosse di salto allora
    controllo le mosse di spostamento.
    
    Esempio 1: [('jump', (3, 5), [(2, 2)])]
    
    Esempio 2: [('move', (5, 1), (-1, -1)), ('move', (5, 1), (-1, 1)), 
    ('move', (5, 3), (-1, -1)), ('move', (5, 3), (-1, 1)), 
    ('move', (5, 5), (-1, -1)), ('move', (5, 5), (-1, 1))]
    
    Questo metodo invoca la funzione jumpingchains (o can_jump) e
    la funzione can_move, che sono definite più avanti. 
    '''
    def actions(self, state):
        moves = []  # inizialmente la lista delle azioni è vuota
        for row in range(8):  # per ogni riga
            for col in range(8):  # per ogni colonna
                if state.board[row][col].lower() == state.to_move:  # se sono il giocatore che deve muovere 
                    for chain in jumpingchains(state.board, (row, col)):  # per ogni elemento della jumpingchains
                        moves.append(('jump', (row, col), chain))  # aggiungo l'elemento nella lista di azioni possibili
        if moves == []:  # moves è vuota ([]) solamente se non ho nessuna pedina da mangiare. In questo caso, posso spostarmi
            for row in range(8):  # per ogni riga
                for col in range(8):  # per ogni colonna
                    if state.board[row][col].lower() == state.to_move:  # se sono il giocatore che deve muovere
                        for delta_row in [-1, 1]:
                            for delta_col in [-1, 1]:
                                # ciclo per ogni coppia (-1,-1), (-1,1), (1,-1), (1,1)
                                if can_move(state.board, (row, col), (delta_row, delta_col)): 
                                    # se posso spostarmi nella direzione selezionata
                                    moves.append(('move', (row, col), (delta_row, delta_col)))
                                    # aggiungo un'azione alla lista
        
        return moves # restituisco le azioni possibili

    
    # metodo che controlla se una mossa (action) è valida dato uno stato (state)
    def is_valid_move(self, state, action) -> bool:
        actions = self.actions(state)
        return action in actions
    
    
    # metodo che applica allo stato corrente (state) una azione (action) e
    # restituisce il nuovo stato aggiornato
    def result(self, state, action):
        type, (from_row, from_col), movements = action
        if type == 'move':
            token_char = state.board[from_row][from_col]
            to_row = from_row+movements[0]
            to_col = from_col+movements[1]
            if token_char == 'x' and to_row == 7:
                token_char = 'X'
            if token_char == 'o' and to_row == 0:
                token_char = 'O'
            board = copy.deepcopy(state.board)
            board[to_row][to_col] = token_char
            board[from_row][from_col] = '-'
        elif type == 'jump':
            start = from_row, from_col
            board = copy.deepcopy(state.board)
            for m in movements:
                start, board = jump(board, start, m)
        state = State(board, opponent(state.to_move))
        return state

Definiamo una classe `Engine` che rappresenta un motore di ricerca nello spazio degli stati. Nel codice seguente viene selezionata una mossa casuale. Le linee di codice per min-max o alpha-beta sono commentate. Quando i metodi saranno implementati, basterà togliere il commento dall'algoritmo da provare.

In [16]:
class Engine:
    # max_depth è la profondità massima di esplorazione
    def __init__(self, max_depth):
        self.max_depth = max_depth
    
    
    # metodo che restituisce la mossa migliore dato uno stato (state)
    def best_move(self, state):
        game = Game()
        
        # algoritmo min-max
        # return minmax_search(state,game,self.max_depth)
        
        # algoritmo alpha-beta
        # return alpha_beta_search(state,game,self.max_depth)
        
        # scelta mossa casuale
        # print(game.to_move(state))
        return random.choice(game.actions(state))

### Algoritmo min-max

Suggerimento: implementare prima min-max e poi alpha-beta.

Alcune funzioni utili in python: [`max(a,b)`](https://docs.python.org/3/library/functions.html#max) (in generale può ricevere altri parametri) che restituisce il più grande tra `a` e `b`.
Esempio:
```
>>> max(3,4)
4
```
Analogamente, [`min(a,b)`](https://docs.python.org/3/library/functions.html#min) restituisce il più piccolo tra `a` e `b`.
Esempio:
```
>>> min(3,4)
3
```
Utilizzare eventualmente max con funzioni lambda (non indispensabile).

In [6]:
def minmax_search(state, game, depth):
    '''
    Metodo che implementa la funzione min-max.
    Suggerimento: devono essere utilizzati i metodi:
    - game.is_terminal: controlla che lo stato sia terminale
    - game.utility: restituisce l'utilità di uno stato per un giocatore
    - game.actions: restituisce la lista di azioni possibili in uno stato
    - game.result: restituisce il risultato dell'applicazione di un'azione in uno stato
    '''

    def max_value(state, depth, player):
        # print(f"player: {player}, depth: {depth}")
        if depth == 0 or game.is_terminal(state):
            return game.utility(game, state, player)
        v = -1000
        actions = game.actions(state)
        for a in actions:
            v = max(v, min_value(game.result(state, a), depth - 1, player))
        return v

    def min_value(state, depth, player):
        # print(f"player: {player}, depth: {depth}")
        if depth == 0 or game.is_terminal(state):
            return game.utility(game, state, player)
        v = 1000
        actions = game.actions(state)
        for a in actions:
            v = min(v, max_value(game.result(state, a), depth - 1, player))
        return v

    # body di minmax_search:
    best = max(game.actions(state), key=lambda a: min_value(game.result(state, a), depth, game.to_move(state)))

    return best


### Algoritmo alpha-beta

Suggerimento: implementare prima min-max e poi alpha-beta.

In [7]:
def alpha_beta_search(state, game, depth):
    '''
    Metodo che implementa la funzione alpha-beta.
    Suggerimento: devono essere utilizzati i metodi:
    - game.is_terminal: controlla che lo stato sia terminale
    - game.utility: restituisce l'utilità di uno stato per un giocatore
    - game.actions: restituisce la lista di azioni possibili in uno stato
    - game.result: restituisce il risultato dell'applicazione di un'azione in uno stato
    '''

    def max_value(state, depth, alpha, beta, player):
        if depth == 0 or game.is_terminal(state):
            return game.utility(state, player)
        v = -1000
        actions = game.actions(state)

        for a in actions:
            v = max(v, min_value(game.result(state, a), depth - 1, alpha, beta, player))
            if v >= beta:
                return v
            alpha = max(alpha,v)
        return v

    def min_value(state, depth, alpha, beta, player):
        if depth == 0 or game.is_terminal(state):
            return game.utility(state, player)
        v = 1000
        actions = game.actions(state)

        for a in actions:
            v = min(v, max_value(game.result(state, a), depth - 1, alpha, beta, player))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body di alpha_beta_pruning
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a), depth, -1000, 1000, game.to_move(state)))